In [1]:
! pip install datasets huggingface emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 17.9 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset

dataset = load_dataset("thesofakillers/jigsaw-toxic-comment-classification-challenge", split="train")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train.csv:   0%|          | 0.00/68.8M [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/60.4M [00:00<?, ?B/s]

test_labels.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/159571 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/306328 [00:00<?, ? examples/s]

In [3]:
import pandas as pd
df = pd.DataFrame(dataset)

In [4]:
df.isnull().sum().sum()

np.int64(0)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   id             159571 non-null  object
 1   comment_text   159571 non-null  object
 2   toxic          159571 non-null  int64 
 3   severe_toxic   159571 non-null  int64 
 4   obscene        159571 non-null  int64 
 5   threat         159571 non-null  int64 
 6   insult         159571 non-null  int64 
 7   identity_hate  159571 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 9.7+ MB


In [6]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [7]:
for col in df.select_dtypes(include="int").columns:
    print(f"\n--- {col} == 1 ---")
    print(df[df[col] == 1])


--- toxic == 1 ---
                      id                                       comment_text  \
6       0002bcb3da6cb337       COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK   
12      0005c987bdfc9d4b  Hey... what is it..\n@ | talk .\nWhat is it......   
16      0007e25b2121310b  Bye! \n\nDon't look, come or think of comming ...   
42      001810bf8c45bf5f  You are gay or antisemmitian? \n\nArchangel WH...   
43      00190820581d90ce           FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!   
...                  ...                                                ...   
159494  fef4cf7ba0012866  "\n\n our previous conversation \n\nyou fuckin...   
159514  ff39a2895fc3b40e                  YOU ARE A MISCHIEVIOUS PUBIC HAIR   
159541  ffa33d3122b599d6  Your absurd edits \n\nYour absurd edits on gre...   
159546  ffb47123b2d82762  "\n\nHey listen don't you ever!!!! Delete my e...   
159554  ffbdbb0483ed0841  and i'm going to keep posting the stuff u dele...   

        toxic  severe_toxic  ob

In [8]:
# 2. Create binary label
df["label"] = ((df[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].sum(axis=1)) > 0).astype(int)



In [9]:
df["label"].value_counts()

,count
label,
0,143346
1,16225


In [10]:
for text in df[df["label"] == 1]["comment_text"][10:20]:
    print(text)
    print("-" * 50)

Why can't you believe how fat Artie is? Did you see him on his recent appearence on the Tonight Show with Jay Leno? He looks absolutely AWFUL! If I had to put money on it, I'd say that Artie Lange is a can't miss candidate for the 2007 Dead pool!   

  
Kindly keep your malicious fingers off of my above comment, . Everytime you remove it, I will repost it!!!
--------------------------------------------------
All of my edits are good.  Cunts like you who revert good edits because you're too stupid to understand how to write well , and then revert other edits just because you've decided to bear a playground grudge, are the problem.  Maybe one day you'll realise the damage you did to a noble project.  201.215.187.159
--------------------------------------------------
Hi! I am back again!
Last warning!
Stop undoing my edits or die!
--------------------------------------------------
Would you both shut up, you don't run wikipedia, especially a stupid kid.
-----------------------------------

In [11]:
import re
import emoji

def clean_text(text):
    # Convert emojis to text (e.g., 😊 -> ":smiling_face_with_smiling_eyes:")
    text = emoji.demojize(text, delimiters=(" ", " "))

    # Lowercase
    text = text.lower()

    # Remove newlines/tabs
    text = re.sub(r'\n|\t', ' ', text)

    # Replace URLs
    text = re.sub(r'http\S+|www\S+', '[URL]', text)

    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)

    # Replace mentions
    text = re.sub(r'@[A-Za-z0-9_]+', '[USER]', text)

    # Keep hashtag word (remove '#' but keep the word)
    text = re.sub(r'#(\w+)', r'\1', text)

    # Normalize repeated punctuation (!!! -> !, ??? -> ?)
    text = re.sub(r'([!?.]){2,}', r'\1', text)

    # Normalize repeated letters (loooove -> loove)
    text = re.sub(r'(.)\1{2,}', r'\1\1', text)

    # Normalize common obfuscations
    text = text.replace("f@ck", "fuck").replace("sh1t", "shit")
    text = text.replace("f#ck", "fuck").replace("s#it", "shit")
    text = text.replace("f*ck", "fuck").replace("s*it", "shit")

    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text


In [12]:
df["clean_text"] = df["comment_text"].apply(clean_text)

In [13]:
df1 = df[["clean_text", "label"]]

In [14]:
for text in df1[df1["label"] == 1]["clean_text"][0:20]:
    print(text)
    print("-" * 50)

cocksucker before you piss around on my work
--------------------------------------------------
hey. what is it. @ | talk . what is it. an exclusive group of some wp talibans.who are good at destroying, self-appointed purist who gang up any one who asks them questions abt their anti-social and destructive (non)-contribution at wp? ask sityush to clean up his behavior than issue me nonsensical warnings.
--------------------------------------------------
bye! don't look, come or think of comming back! tosser.
--------------------------------------------------
you are gay or antisemmitian? archangel white tiger meow! greetingshh! uh, there are two ways, why you do erased my comment about ww2, that holocaust was brutally slaying of jews and not gays/gypsys/slavs/anyone. 1 - if you are anti-semitian, than shave your head bald and go to the skinhead meetings! 2 - if you doubt words of the bible, that homosexuality is a deadly sin, make a pentagram tatoo on your forehead go to the satanistic 

In [15]:
df1.to_csv("toxic_word_dataset.csv", index=False)

In [16]:
df1["label"].value_counts()

,count
label,
0,143346
1,16225


In [17]:
from sklearn.utils import resample

df_majority = df1[df1["label"]==0]
df_minority = df1[df1["label"]==1]

df_majority_ds = resample(
    df_majority,
    replace=False,
    n_samples=len(df_minority),
    random_state=42
)

# df_minority_ds = resample(
#     df_minority,
#     replace=False,
#     n_samples=10000,
#     random_state=42
# )

df_balanced = pd.concat([df_majority_ds, df_minority])

# Shuffle the dataset
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

print("Class distribution after undersampling:")
print(df_balanced.label.value_counts())


Class distribution after undersampling:
label
0    16225
1    16225
Name: count, dtype: int64


In [18]:
df_balanced.head()

,clean_text,label
0,we want to keep the first sentence of the lead...,0
1,who cares. goethean is a capuchin monkeys anyway,1
2,yes i would say the book source is enough to a...,0
3,thanks for your help. i guess i will have to s...,0
4,then do the same for the tiger accounts.,0


In [19]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df_balanced["clean_text"],
    df_balanced["label"],
    test_size=0.2,
    stratify=df_balanced["label"],
    random_state=42
)

In [20]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

train_encodings = tokenizer(
    list(X_train),
    truncation=True,
    padding=True,
    max_length=128
)

test_encodings = tokenizer(
    list(X_test),
    truncation=True,
    padding=True,
    max_length=128
)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [21]:
import torch
import numpy as np

class ToxicCommentsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            key: torch.tensor(val[idx]) for key, val in self.encodings.items()
        } | {
            "labels": torch.tensor(self.labels[idx])
        }

y_train = y_train.reset_index(drop=True)  # if it's a pandas Series
y_test = y_test.reset_index(drop=True)

# Or convert to numpy
y_train = np.array(y_train)
y_test = np.array(y_test)

train_dataset = ToxicCommentsDataset(train_encodings, y_train)
test_dataset = ToxicCommentsDataset(test_encodings, y_test)

print("Train size:", len(train_dataset))
print("Test size:", len(test_dataset))

Train size: 25960
Test size: 6490


In [25]:
from transformers import TrainingArguments, Trainer, AutoModelForSequenceClassification, EarlyStoppingCallback
import torch
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

# ----- Load model -----
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
import os

os.environ["WANDB_DISABLED"] = "true"

# ----- Metrics -----
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary")
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

# ----- Training args -----
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",   # (was eval_strategy -> should be evaluation_strategy)
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
    load_best_model_at_end=True,
    fp16=True,
    report_to="none",

)

# ----- Trainer -----
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,   # use your balanced dataset
    eval_dataset=test_dataset,
    tokenizer=tokenizer,                     # (was processing_class, correct is tokenizer)
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=1)]

)

# ----- Train -----
trainer.train()

# ----- Evaluate -----
eval_results = trainer.evaluate()
print("Eval results:", eval_results)


/tmp/ipython-input-1049065218.py:37: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.182200,0.162439,0.941140,0.941685,0.940524,0.941104
2,0.127100,0.224405,0.934515,0.907986,0.967026,0.936577


Eval results: {'eval_loss': 0.1624387949705124, 'eval_accuracy': 0.9411402157164869, 'eval_precision': 0.9416846652267818, 'eval_recall': 0.9405238828967643, 'eval_f1': 0.9411039161270429, 'eval_runtime': 7.8812, 'eval_samples_per_second': 823.479, 'eval_steps_per_second': 103.03, 'epoch': 2.0}


In [27]:
# Save the fine-tuned model
trainer.save_model("./toxic_model")

# Save tokenizer too
tokenizer.save_pretrained("./toxic_model")

('./toxic_model/tokenizer_config.json',
 './toxic_model/special_tokens_map.json',
 './toxic_model/vocab.txt',
 './toxic_model/added_tokens.json',
 './toxic_model/tokenizer.json')